In [1]:
import torch

In [2]:
# Tensor porduct와 모든 element를 한번에 더하기 Toy example

# 두 개의 텐서 A와 B 생성
X = torch.tensor([[1, 2], [3, 4],[5,8]])
Y = torch.tensor([[5, 6], [7, 8]])

# Kronecker product 계산
kron_Xi_10 = torch.kron(X, Y)

# Kronecker product의 모든 요소를 합산
result = torch.sum(kron_result)

print(result)  # 결과 출력

tensor(598)


In [3]:
import torch.distributions as dist

# 시드 설정 (재현 가능한 결과를 위해)
torch.manual_seed(0)

# 첫 번째 데이터셋: 상관관계 없는 5차원 다변량 정규 분포
mean1 = torch.zeros(5)
cov1 = torch.eye(5)

# 두 번째 데이터셋: 마지막 두 차원 간의 상관관계가 0.8인 다변량 정규 분포
mean2 = torch.zeros(5)
cov2 = torch.eye(5)
cov2[3, 4] = cov2[4, 3] = 0.8

# 데이터 생성 (radom forest 논문과 같은 세팅)
X = dist.MultivariateNormal(mean1, cov1).sample((1000,))
Y = dist.MultivariateNormal(mean2, cov2).sample((1000,))

print("Data 1 Shape:", X.shape)  # (300, 5)
print("Data 2 Shape:", Y.shape)  # (300, 5)


Data 1 Shape: torch.Size([1000, 5])
Data 2 Shape: torch.Size([1000, 5])


In [4]:
def Pdist2(x, y):
    """compute the paired distance between x and y."""
    x_norm = (x ** 2).sum(1).view(-1, 1)
    if y is not None:
        y_norm = (y ** 2).sum(1).view(1, -1)
    else:
        y = x
        y_norm = x_norm.view(1, -1)
    Pdist = x_norm + y_norm - 2.0 * torch.mm(x, torch.transpose(y, 0, 1))
    Pdist[Pdist<0]=0
    return Pdist

In [5]:
Dxx = Pdist2(X, X)
Dyy = Pdist2(Y, Y)
Dxy = Pdist2(X, Y)
sigma0 = 1.0

In [6]:
Kxx = torch.exp(-(Dxx / sigma0))
Kyy = torch.exp(-(Dyy / sigma0))
Kxy = torch.exp(-(Dxy / sigma0))

In [7]:
m,n = Kxy.shape #smaple size
print("m : ",m)
print("n : ",n)

m :  1000
n :  1000


MMD 구하기

In [8]:
def mmd() :
    Dxx = Pdist2(X, X)
    Dyy = Pdist2(Y, Y)
    Dxy = Pdist2(X, Y)
    Kxx = torch.exp(-(Dxx / sigma0))
    Kyy = torch.exp(-(Dyy / sigma0))
    Kxy = torch.exp(-(Dxy / sigma0))
    term1 = torch.sum(Kxx - torch.diag(torch.diag(Kxx))) / (m * (m - 1))
    term2 = torch.sum(Kyy - torch.diag(torch.diag(Kyy))) / (n * (n - 1))
    term3 = -2 * torch.sum(Kxy) / (m * n)

    mmd_sq = term1 + term2 + term3
    return mmd_sq
    
mmd()

tensor(0.0030)

In [9]:
C1 = torch.trace(Kxx.T@Kxx)
C2 = torch.sum(Kxx@Kxx)
C3 = torch.sum(torch.kron(Kxx, Kxx))
C4 = torch.sum(torch.kron(Kxx, Kyy))
C5 = torch.sum(Kxx @ Kxy)
C6 = torch.sum(torch.kron(Kxx,Kxy))
C7 = torch.sum(Kxy@Kyy)
C8 = torch.sum(torch.kron(Kxy,Kyy))
C9 = torch.trace(Kxy.T @ Kxy)
C10 = torch.sum(Kxy.T @ Kxy)
C11 = torch.sum(Kxy @ Kxy.T)
C12 = torch.sum(torch.kron(Kxy,Kxy))
C13 = torch.trace(Kyy.T @ Kyy)
C14 = torch.sum(Kyy@Kyy)
C15 = torch.sum(torch.kron(Kyy,Kyy))

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 4000000000000 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
# 2개
n2 = n**2
mn = m*n
m2 = m**2
# 3개
m3 = m**3
m2n = m**2 * n
mn2 = m * (n**2)
n3 = n**3
# 4개
m4 = m**4
m3n = (m**3) * n
m2n2 = (m**2) * (n**2)
mn3 = m* (n**3)
n4 = n**4

In [ ]:
mmd2 = mmd() **2

In [ ]:
def Xi_10() :
    result = n4 * C2
    result += 2 * mn * n2 * C4
    result -= 2 * mn * n3 * C5
    result -= 2 * n3 * C6
    result -= 4 * m2n * C8
    result += m2n * n2 * C10
    result += 3 * mn * n2 * C12
    result += m3 * C15
    return result/((m**3)*(n**4))-mmd2
Xi_10()

tensor(0.0765)

In [ ]:
def Xi_01() :
    result = n3 * C3
    result += 2 * m2n * C4
    result -= 4 * mn2 * C6
    result -= 2 * m3n * C7
    result -= -2 * m3 * C8
    result += m2n2 * C11
    result += 3 * m2n * C12
    result += m4 * C14 
    
    return result/((m**4)*(n**3))-mmd2

Xi_01()

tensor(0.0246)

In [ ]:
def Xi_11() :
    result =  n3 * C2
    result += 2 * mn * C4
    result -= 2 * mn2 * C5
    result -= 2 * n2 * C6
    result -= 2 * m2n *  C7
    result -= 2 * m2 *  C8
    result += (1/4) * m2n2 * C9
    result += (3/4) * m2n * C10
    result += (3/4) * mn2 * C11
    result += (9/4) * mn *C12
    result +=  m3 *C14
    
    return result/((m**3)*(n**3))-mmd2

Xi_11()

tensor(0.0134)

In [ ]:
def Xi_20() :
    result =  n4 * C1
    result += 2 * n2 * C4
    result -= 4 * n3 * C5
    result -= 4 * mn * C8
    result += 2 * mn2 *  C10
    result += 2 * n2 *  C12
    result +=  m2 * C15
    
    return result/((m**2)*(n**4))-mmd2
Xi_20()

tensor(0.0204)

In [ ]:
def Xi_02() :
    result =  n2 * C3
    result += 2 * m2 * C4
    result -= 4 * mn * C6
    result -= 4 * m3 * C7
    result += 2 * m2n *  C11
    result += 2 * m2 *  C12
    result +=  m4 * C13
    
    return result/((m**4)*(n**2))-mmd2
Xi_02()

tensor(0.1040)

In [ ]:
def Xi_12() :
    result =  mn2 * C2
    result += 2 * m * C4
    result -= 2 * mn * C5
    result -= 2 * n * C6
    result -= 4 * m2 *  C7
    result += (1/2) * m2n *  C9
    result += (1/2) * m2 * C10
    result += (3/2) * mn *  C11
    result += (3/2) * m *  C12
    result +=  m3 * C13
    
    return result/((m**3)*(n**2))-mmd2
Xi_12()

tensor(0.2368)

In [ ]:
def Xi_21() :
    result =  n3 * C1
    result += 2 * n * C4
    result -= 4 * n2 * C5
    result -= 2 * mn * C7
    result -= 2 * m *  C8
    result += (1/2) * mn2 *  C9
    result += (3/2) * mn * C10
    result += (1/2) * n2 *  C11
    result += (3/2) * n *  C12
    result +=  m2 * C14
    
    return result/((m**2)*(n**3))-mmd2
Xi_21()

tensor(0.0226)

In [ ]:
def Xi_22() :
    result =  n2 * C1
    result += 2 * C4
    result -= 4 * n * C5
    result -= 4 * m * C7
    result += mn * C9
    result += m * C10
    result += n * C11
    result += C12
    result += m2 * C13

    
    return result/((m**2)*(n**2))-mmd2
Xi_22()

tensor(0.1164)

Variance estimator
$$
\begin{align*}
\therefore 
\text{Var}(U_{mn}) = &
{4(m-2)(m-3)(n-2)\over m(m-1)n(n-1)}\xi_{0,1} + {2(n-2)(n-3)\over m(m-1)n(n-1)}\xi_{0,2} \\
&+
{4(m-2)(n-2)(n-3)\over m(m-1)n(n-1)}\xi_{1,0} + 
{16(n-2)(m-2)\over m(m-1)n(n-1)}\xi_{1,1} \\
&+ {8(m-2)\over m(m-1)n(n-1)}\xi_{1,2} +
{2(n-2)(n-3) \over m(m-1)n(n-1)}\xi_{2,0} \\
&+ {2(n-2) \over m(m-1)n(n-1)} \xi_{2,1} + {4\over m(m-1)n(n-1)}\xi_{2,2}
\end{align*}
$$

In [ ]:
def mmd(X,Y) :
    Dxx = Pdist2(X, X)
    Dyy = Pdist2(Y, Y)
    Dxy = Pdist2(X, Y)
    Kxx = torch.exp(-(Dxx / sigma0))
    Kyy = torch.exp(-(Dyy / sigma0))
    Kxy = torch.exp(-(Dxy / sigma0))
    term1 = torch.sum(Kxx - torch.diag(torch.diag(Kxx))) / (m * (m - 1))
    term2 = torch.sum(Kyy - torch.diag(torch.diag(Kyy))) / (n * (n - 1))
    term3 = -2 * torch.sum(Kxy) / (m * n)

    mmd_sq = term1 + term2 + term3
    return mmd_sq
    
mmd(X,Y)

tensor(0.0075)

In [ ]:
def est_var():
    term1 = (4 * (m - 2) * (m - 3) * (n - 2) / (m * (m - 1) * n * (n - 1))) * Xi_01()
    term2 = (4 * (m - 2) * (n - 2) * (n - 3) / (m * (m - 1) * n * (n - 1))) * Xi_10()
    term3 = (16 * (n - 2) * (m - 2) / (m * (m - 1) * n * (n - 1))) * Xi_11()
    term4 = (2 * (n - 2) * (n - 3) / (m * (m - 1) * n * (n - 1))) * Xi_02()
    term5 = (2 * (n - 2) * (n - 3) / (m * (m - 1) * n * (n - 1))) * Xi_20()
    term6 = (8 * (m - 2) / (m * (m - 1) * n * (n - 1))) * Xi_12()
    term7 = (2 * (n - 2) / (m * (m - 1) * n * (n - 1))) * Xi_21()
    term8 = (4 / (m * (m - 1) * n * (n - 1))) * Xi_22()

    var = term1 + term2 + term3 + term4 + term5 + term6 + term7 + term8

    return var

est_var()


tensor(0.0097)

In [ ]:
# MMD를 추정할 횟수
num_samples = 10000

# MMD 값들을 저장할 배열
mmd_samples = torch.zeros(num_samples)


# Monte Carlo 시뮬레이션을 수행하여 MMD 값을 추정합니다.
for i in range(num_samples):
    # 새로운 데이터를 생성하고
    X = dist.MultivariateNormal(mean1, cov1).sample((200,))
    Y = dist.MultivariateNormal(mean2, cov2).sample((10,))

    
    # MMD 값을 계산하고 결과를 리스트에 추가합니다.
    mmd_value = mmd(X,Y)
    mmd_samples[i] = mmd_value

# MMD의 추정된 분산 계산
mmd_variance = torch.var(mmd_samples, unbiased=True)

print("Estimated Variance of MMD:", mmd_variance)


Estimated Variance of MMD: tensor(0.0001)
